In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.inquiry import Inquiry
from libs.alibaba.inquiry import element_has_css_class
from libs.email import Email
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from string import Template
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import time
import re
import math

import types
import traceback
import threading

import pendulum

import logging
from logging.handlers import TimedRotatingFileHandler

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
market

. storage markets.json
.\storage\markets.json was deserialized!


{'name': 'Eyelashes',
 'directory': 'E:\\glitter\\Eyelashes',
 'lid': 'eyelash@glittergroupcn.com',
 'lpwd': 'Wangzhen68970747',
 'accounts': [{'lid': 'eyelash1@glittergroupcn.com',
   'lpwd': 'Zhangying25160',
   'lname': 'Emily Zhang',
   'disabled': True,
   'mobile': '15615125160'},
  {'lid': 'eyelash2@glittergroupcn.com',
   'lpwd': 'Huyinping18130',
   'lname': 'Ada Alva',
   'disabled': False,
   'mobile': '18563918130'}],
 'homepage': 'https://glittereyelash.en.alibaba.com',
 'lname': 'Jeremy Wang',
 'mobile': '17685593100'}

In [ ]:
account = market['accounts'][1]
print(account)
inquiry = Inquiry(market, account, headless=False)

In [3]:
inquiry = Inquiry(market, headless=False)

E:\glitter\Eyelashes_config [] inquiry_tracking_ids_Jeremy.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids_Jeremy.json was deserialized!


In [4]:
def next_page(self):
    try:
        next_btn = self.browser.find_element_by_css_selector('div.ui2-pagination a.next')
        
        self.click(next_btn)
        
        div = WebDriverWait(self.browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.aui-loading')))
        WebDriverWait(self.browser, 15).until(EC.staleness_of(div))
        return True
    except NoSuchElementException as e:
        return False
    
inquiry.next_page = types.MethodType(next_page, inquiry)

In [5]:
inquiry.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [ ]:
inquiry.load_url()

In [31]:
inquiry.load_url()
time.sleep(1)
inquiry_list = []
while True:
    inquiries = inquiry.get_inquiries()
    inquiry_list = inquiry_list + inquiries
    time.sleep(1)
    for enquiry in inquiries:
        try:
            inquiry.open_inquiry_in_new_tab(enquiry)
            conversation = inquiry.get_conversation()
            enquiry['conversation'] = conversation
        except Exception as e:
            traceback.print_exc()
        finally:
            inquiry.close_inquiry_and_switch_back()
            
    if inquiry.next_page():
        continue
        
    break
len(inquiry_list)

156

In [43]:
enquiries = []
idx = 0
for enquiry in inquiry_list:
    if '【RFQ】 Request for Quotation' in enquiry['tags']:
        continue
    if 'prestl bernhard' in enquiry['buyer']:
        continue
        
    first_message = enquiry['conversation']['messages'][0]
    if first_message['position'] == 'right':
        continue
        
    idx += 1
    comming_datetime = pendulum.parse(first_message['date'], tz='Asia/Shanghai')
    enquiry['comming_datetime'] = comming_datetime
#     print(idx, enquiry['id'], comming_datetime, enquiry['buyer_country'], enquiry['buyer'])
    enquiries.append(enquiry)
len(enquiries)

143

In [ ]:
enquiries.sort(key=lambda eq:eq['comming_datetime'])

In [66]:
length = len(enquiries)
count = 0
for idx, enquiry in enumerate(enquiries):
    enquiry['duplicated'] = False
    offset = 1
    country = enquiry['buyer_country']
    buyer = enquiry['buyer']
    cdt = enquiry['comming_datetime'].subtract(hours=3)
    while idx-offset >= 0:
        dt = enquiries[idx-offset]['comming_datetime']
        if dt < cdt:
            break
    
        if country == enquiries[idx-offset]['buyer_country'] and buyer == enquiries[idx-offset]['buyer']:
            enquiry['duplicated'] = True

        offset += 1

    if not enquiry['duplicated']:
        count += 1
        
    print(idx, count, enquiry['id'], enquiry['duplicated'], enquiry['comming_datetime'], enquiry['buyer_country'], enquiry['buyer'])


0 1 100669100140 False 2018-05-13T04:41:00+08:00 United Kingdom Firdaous Sahli
1 2 100723764260 False 2018-05-14T03:31:00+08:00 United States Breanna Holliday
2 3 100674736381 False 2018-05-23T19:15:00+08:00 Switzerland reza dubravac
3 4 100676237542 False 2018-05-25T22:52:00+08:00 Italy Maria Spina
4 5 100685999010 False 2018-06-15T23:11:00+08:00 United States Kendra Jenkins
5 6 100687438364 False 2018-06-21T03:55:00+08:00 United Kingdom Angela Pham
6 7 100743144152 False 2018-06-22T02:27:00+08:00 United States Kimberly Hardy
7 8 100743156131 False 2018-06-22T02:49:00+08:00 France Maddy GRUPPO
8 9 100690268929 False 2018-06-22T05:03:00+08:00 United States Scott Godar
9 10 100681681922 False 2018-06-22T08:36:00+08:00 Australia Kelly Hughes
10 11 100689148256 False 2018-06-23T02:11:00+08:00 Belize Joan Crooks
11 12 100682197401 False 2018-06-23T07:08:00+08:00 France Reguina cheloukhina
12 13 100690877082 False 2018-06-26T05:30:00+08:00 United States Cecilia Covarrubias-Hernandez
13 14 1

In [145]:
def search(inquiries, st, et):
    result = []
    for enquiry in inquiries:
        if enquiry['duplicated']:
            continue
        if enquiry['buyer_country'] != 'United States':
            continue
        dt = enquiry['comming_datetime']
        stime = dt.set(hour=int(st.split(':')[0]), minute=int(st.split(':')[1]))
        etime = dt.set(hour=int(et.split(':')[0]), minute=int(et.split(':')[1]))
        if dt >= stime and dt <etime:
#             print(enquiry['comming_datetime'], stime, etime)
            result.append(enquiry)
    return result

In [117]:
st = '08:00'
et = '09:00'
result = search(enquiries, st, et)
len(result)

5

In [146]:
dt = pendulum.parse('15:00:00')
results = []
for i in range(144):
    st = dt.to_time_string()
    dt = dt.add(minutes=10)
    et = dt.to_time_string()
    result = search(enquiries, st, et)
    results.append([i, st, et, result])
    print(len(result), end='\t')

1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	1	1	0	0	0	0	0	1	0	1	0	2	3	0	2	0	0	0	0	0	3	0	0	1	1	1	0	2	1	0	2	0	0	1	1	0	2	0	0	2	1	2	2	1	0	2	1	0	0	0	1	1	1	1	0	1	0	0	0	0	0	0	0	0	1	0	0	0	0	1	2	0	1	0	1	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	0	0	0	1	1	

In [148]:
result = results[60]
print(result[0], result[1], result[2])
for enquiry in result[3]:
    print(enquiry['id'], enquiry['comming_datetime'], enquiry['buyer_country'], enquiry['buyer'])

60 01:00:00 01:10:00
100767996234 2018-08-04T01:08:00+08:00 United States Nayla Payton
100715413892 2018-08-16T01:01:00+08:00 United States Ghazele Alalouch
100721366664 2018-08-19T01:03:00+08:00 United States C J


In [155]:
count = 0
counts = {}
for enquiry in enquiries:
    if enquiry['duplicated']:
        continue
        
    count += 1
    country = enquiry['buyer_country']
        
    if country not in counts:
        counts[country] = 0
        
    counts[country] += 1
count_list = []
for country in counts:
    count_list.append([country, counts[country]])
count_list.sort(reverse=True, key=lambda x: x[1])

In [165]:
for item in count_list:
    print(format(item[1]/count*100, '0.2f')+'%', end='\t')
    print(item[0], end='\t')
    print(item[1], end='\t')
    print()

45.16%	United States	56	
9.68%	United Kingdom	12	
8.87%	Peru	11	
3.23%	Canada	4	
2.42%	Australia	3	
2.42%	Mexico	3	
2.42%	Colombia	3	
1.61%	Italy	2	
1.61%	France	2	
1.61%	Poland	2	
1.61%	Malaysia	2	
1.61%	United Arab Emirates	2	
1.61%	Zambia	2	
1.61%	South Africa	2	
1.61%	Egypt	2	
0.81%	Switzerland	1	
0.81%	Belize	1	
0.81%	Kyrgyzstan	1	
0.81%	India	1	
0.81%	Belgium	1	
0.81%	Sweden	1	
0.81%	Estonia	1	
0.81%	Panama	1	
0.81%	Bulgaria	1	
0.81%	Bahrain	1	
0.81%	Saudi Arabia	1	
0.81%	Qatar	1	
0.81%	Brazil	1	
0.81%	Namibia	1	
0.81%	Ecuador	1	
0.81%	Portugal	1	


In [ ]:
inquiry.browser.switch_to.default_content()

In [ ]:
inquiry.load_url()

In [ ]:
threads = inquiry.webww_check()

In [ ]:
dialog_iframe = inquiry.browser.find_element_by_css_selector('#webatm2-iframe')
inquiry.browser.switch_to.frame(dialog_iframe)
sidebar = inquiry.browser.find_ele.ment_by_css_selector('.webatm2-sidebar')
head = inquiry.browser.find_element_by_css_selector('.webatm2-window-head')
body = inquiry.browser.find_element_by_css_selector('.webatm2-window-body')


In [ ]:
thread = threads[0]
thread

In [ ]:
inquiry.webww_switch_to_thread(threads[1], sidebar, head, body)

In [ ]:
inquiry.browser.switch_to.default_content()

In [ ]:
for t in threads:
    inquiry.webww_switch_to_thread(t, sidebar, head, body)

In [ ]:
not {}

In [ ]:
inquiry.load_reply_templates()

In [ ]:
inquiry.reply_templates

In [ ]:
def get_inquiries(self):
    html = pq(self.browser.page_source)
    inquiry_list = html.find('.ui2-list-body .aui2-grid-wraper')
    inquiries = []
    for elem in inquiry_list:
        div = pq(elem)
        item = {}
        item['url'] = div.find('a').attr('href')
        text = div.find('div.spec-inquiry-id').text().strip()
        item['id'] = re.search('(\d+)', text).group(1)
        item['date'] = div.find('div.spec-inquiry-id+div').text().strip()
        item['tags'] = []
        for span in div.find('div.aui-tags span'):
            item['tags'].append(pq(span).text())
        item['is_replied'] = True if div.find('td:nth-child(2) i[title="已回复"]') else False
        item['title'] = div.find('td:nth-child(3)').text()
        item['buyer'] = div.find('td:nth-child(4)').text().strip()
        item['is_buyer_online'] = True if div.find('td:nth-child(5) div.atm-online') else False
        text = div.find('td:nth-child(6) i').attr('title')
        try:
            item['buyer_country'] = re.sub(' \[.*\]', '', text)
            item['buyer_local_time'] = re.search('\[Local Time: (.*)\]', text).group(1)
        except AttributeError:
            item['buyer_country'] = text
            item['buyer_local_time'] = None
        item['responsible_person'] = div.find('td:nth-child(7)').text()
        item['status'] = div.find('td:nth-child(8)').text()
        if '新询盘' in item['status'] or 'New Inquiry' in item['status']:
            item['is_new'] = True
        else:
            item['is_new'] = False
        inquiries.append(item)
    return inquiries

inquiry.get_inquiries = types.MethodType(get_inquiries, inquiry)

In [ ]:
def send_message(self, message, attach=None):
    chat_form = self.browser.find_element_by_css_selector('form.reply-wrapper')
    chat_form.find_element_by_css_selector('div.holder').click()

    uploading_failed = False
    if attach:
        file_input = self.browser.find_element_by_css_selector('input#attachs')
        file_input.send_keys(attach)

        css = 'div.next-upload-list-item:last-child'
        div = WebDriverWait(self.browser, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css)))
        try:
            WebDriverWait(self.browser, 150).until(
                element_has_css_class(div, 'next-upload-list-item-done'))
        except TimeoutException:
            print('time out, cancel uploading')
            div.find_element_by_css_selector('i.next-icon-close').click()
            uploading_failed = True

    if uploading_failed:
        return False

    # if uploading_failed or attach is None:
    #     doc = pq(message)
    #     doc.find('span.attach').remove()
    #     message = doc.outer_html()

    WebDriverWait(self.browser, 15).until(
        EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, '#inquiry-content_ifr')))
    body = self.browser.find_element_by_tag_name('body')
    self.click(body)

    js = self.reply_js_template.format(message=message)
    self.browser.execute_script(js)
    self.browser.switch_to.default_content()

    time.sleep(1)

    btn_send = chat_form.find_element_by_css_selector('button.send')
    btn_send.click()
    
    return True

inquiry.send_message = types.MethodType(send_message, inquiry)

In [ ]:

def reply(self, enquiry):
    try:
        self.open_inquiry_in_new_tab(enquiry)
        conversation = self.get_conversation()

        messages = conversation['messages']
        buyer = conversation['buyer']

        last_buyer_messages = []
        is_replied = False
        reply_count = 0
        for msg in messages:
            if msg['position'] == 'left':
                last_buyer_messages.append(msg)
                is_replied = False
            elif msg['position'] == 'right' and 'Seller Assistant' not in msg['user']:
                last_buyer_messages = []
                is_replied = True
                reply_count += 1

        # check if email address is already exists in conversation
        text = ''
        for msg in last_buyer_messages:
            text = text + '\n\n ' + msg['content']
        emails = re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)

        if buyer['email']:
            emails.append(buyer['email'])

        eid = enquiry['id']
        tracking = self.tracking_ids[eid]
        last_status = tracking['status'][-1]
        greetings = '<p>Pleased to hear from you.</p><br>'
        if last_status == 'new':
            if emails:
                mime_message = Email.message_of_product_catalog(self.market, self.account, buyer['name'])
#                 if Email.send(self.account, emails, mime_message):
                if True:
                    params = {'buyer': buyer['name'], 'greetings': greetings, 'email': ','.join(emails), 'sender': self.lname}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    if self.send_message(message, self.catalog):
                        tracking['status'].append('catalog was sent by email')
                        self.save_tracking_ids()
                    else:
                        tracking['status'].append('catalog was sent by email, but reply was failed')
                        self.save_tracking_ids()
            else:
                params = {'buyer': buyer['name'], 'greetings': greetings, 'sender': self.lname}
                message = self.reply_templates['notify_first_reply_without_email'].substitute(params)
                if self.send_message(message, self.catalog):
                    tracking['status'].append('catalog was sent directly')
                    self.save_tracking_ids()
        elif 'catalog was sent' in last_status and 'reply was failed' in last_status:
            if is_replied:
                tracking['status'].append('catalog was sent by email')
                self.save_tracking_ids()
            else:
                params = {'buyer': buyer['name'], 'greetings': greetings, 'email': ','.join(emails), 'sender': self.lname}
                message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                if self.send_message(message, self.catalog):
                    tracking['status'].append('catalog was sent by email')
                    self.save_tracking_ids()

        elif 'catalog was sent' in last_status:
            pass
        elif 'wait for email address' in last_status:
            text = ''
            for msg in last_buyer_messages:
                text = text + '\n\n ' + msg['content']
            emails = re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)
            print(emails)
            if emails:
                tracking['emails'] = emails
                self.save_tracking_ids()
                mime_message = Email.message_of_product_catalog(self.market, self.account, buyer['name'])
                if Email.send(self.account, emails, mime_message):
                    params = {'buyer': buyer['name'], 'greetings': '', 'email': ', '.join(emails),
                              'sender': self.lname}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    if self.send_message(message, self.catalog):
                        tracking['status'].append('catalog was sent by email')
                        self.save_tracking_ids()
    except Exception as e:
        self.notify("danger", '回复 询盘 [' + enquiry['id'] + '] 时 发生错误! ' + str(e))
        traceback.print_exc()
    finally:
        self.close_inquiry_and_switch_back()
inquiry.reply = types.MethodType(reply, inquiry)

In [ ]:
inquiry.load_url()
time.sleep(0.1)

In [ ]:
inquiries = inquiry.get_inquiries()

In [ ]:
enquiry = inquiries[3]
enquiry

In [ ]:
inquiry.is_auto_reply_needed(enquiry)

In [ ]:
inquiry.reply(enquiry)

In [ ]:
inquiry.account

In [ ]:
inquiry.load_reply_templates()
inquiry.reply_templates['notify_first_reply_without_email']

In [ ]:
inquiry.tracking_ids['11947313067']['status'] = ['new']
inquiry.tracking_ids

In [ ]:
inquiry.check()

In [ ]:
inquiry.check()

In [ ]:
inquiry.tracking_ids

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
inquiry.load_find_catalog()

In [ ]:
inquiry.catalog

In [ ]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
inquiry = Inquiry(market)

In [ ]:
emails = ['changshu.qd@gmail.com', 'odingdongo@hotmail.com']
mime_message = Email.message_of_product_catalog(inquiry.market, inquiry.account, 'prestl bernhard')
Email.send(inquiry.account, emails, mime_message)

In [ ]:
inquiry.account

In [ ]:
enquiry = inquiries[8]
enquiry

In [ ]:
enquiry = inquiries[8]
inquiry.open_inquiry_in_new_tab(enquiry)
conversation = inquiry.get_conversation()
inquiry.close_inquiry_and_switch_back()
messages = conversation['messages']
buyer = conversation['buyer']
messages

In [ ]:
enquiry = inquiries[8]
enquiry

In [ ]:
# inquiry.load_tracking_ids()
inquiry.tracking_ids

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
# inquiry.close_inquiry_and_switch_back()
# inquiry.tracking_ids['11947313067']['status'] = ['new']
enquiry = inquiries[0]
inquiry.reply(enquiry)

In [ ]:
inquiry.tracking_ids

In [ ]:
def try_login_webww():
    btn_login = WebDriverWait(inquiry.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-iconbar')))
    btn_login.click()

In [ ]:
panel = inquiry.browser.find_element_by_css_selector('#webatm2-panel')

In [ ]:
inquiries[11]

In [ ]:
re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)

In [ ]:
inquiry.browser.save_screenshot("screenshot.png")

In [ ]:
text = 'Inquiry ID：100708962003'

In [ ]:
text = '询价单号：100708962003'

In [ ]:
re.search('(\d+)', text).group(1)

In [ ]:
text

In [ ]:
msg = 'ldskfjsl'
js = "document.querySelector('.webatm2-window-body .webatm2-editor-body').innerHTML = `{message}`;"
js.format(message=msg)

In [ ]:
from string import Template

In [ ]:
params = {'salutation': 'chang', 'link': 'link', 'sender': 'shu'}
templ = Template('$salutation, lsdkfjlskf. $sender')
msg = templ.substitute(params)

In [ ]:
msg